In [2]:
!pip install pymysql

In [1]:
import pymysql

In [52]:
class DataBase:
    def __init__(self):
        self.connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "teleton"
        )
        self.cursor = self.connection.cursor()
    
        print("Conexión exitosa")
    
    def insert(self, tabla, nomina, nombre, puesto, tags, certificado, estatus):
            sql = f"INSERT INTO {tabla} (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('{nomina}', '{nombre}', '{puesto}', '{tags}', '{certificado}', '{estatus}')"
            print(sql)
            try:
                self.cursor.execute(sql)
                print(f"{nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def select(self, tabla, what, _from, where, value):
        if what == "*":
            sql = f"SELECT {what} FROM {_from}"
        else:
            sql = f"SELECT {what} FROM {_from} where {where} = '{value}'"
        print(sql)
        self.cursor.execute(sql)
        val = self.cursor.fetchall()
        return val
        
    def update(self, tabla, what, value, where_col, where_val):
        sql = f"UPDATE {tabla} set {what} = '{value}' where {where_col} = '{where_val}'"
        self.cursor.execute(sql)
        print("Actualización exitosa")
        
    def delete(self, tabla, where, value):
        sql = f"DELETE FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        print("Eliminación exitosa")

database = DataBase()

Conexión exitosa


In [29]:
database.insert("users","A0175068", "Jairo", "Estudiante", "IDM", "pdjsnem", "Activo")

INSERT INTO users (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('A0175068', 'Jairo', 'Estudiante', 'IDM', 'pdjsnem', 'Activo')
Jairo ha sido añadido a la base de datos.


In [56]:
database.select(tabla="users", what = "*", _from = "users", where = "Nomina", value = "A0175043")

SELECT * FROM users


(('A0175043', 'enrique', 'Estudiante', 'IDM', 'gshdjhnbsjcjdnlksd', 'Activo'),
 ('A0175068', 'Jairo', 'Estudiante', 'IDM', 'pdjsnem', 'Activo'))

In [55]:
database.update(tabla = "users", what = "Nombre", value="Juanito", where_col = "Nomina", where_val = "A01752068")

Actualización exitosa


In [53]:
database.delete(tabla = "users", where = "Nomina", value="A01752068")

Eliminación exitosa


In [80]:
from OpenSSL import crypto, SSL

def cert_gen(
    emailAddress="emailAddress",
    commonName="commonName",
    countryName="NT",
    localityName="localityName",
    stateOrProvinceName="stateOrProvinceName",
    organizationName="organizationName",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds=0,
    validityEndInSeconds=10*365*24*60*60,
    KEY_FILE = "private.key",
    CERT_FILE="selfsigned.crt"):
    #can look at generated file using openssl:
    #openssl x509 -inform pem -in selfsigned.crt -noout -text
    # create a key pair
    k = crypto.PKey()
    k.generate_key(crypto.TYPE_RSA, 4096)
    # create a self-signed cert
    cert = crypto.X509()
    cert.get_subject().C = countryName
    cert.get_subject().ST = stateOrProvinceName
    cert.get_subject().L = localityName
    cert.get_subject().O = organizationName
    cert.get_subject().OU = organizationUnitName
    cert.get_subject().CN = commonName
    cert.get_subject().emailAddress = emailAddress
    cert.set_serial_number(serialNumber)
    cert.gmtime_adj_notBefore(0)
    cert.gmtime_adj_notAfter(validityEndInSeconds)
    cert.set_issuer(cert.get_subject())
    cert.set_pubkey(k)
    cert.sign(k, 'sha512')
    with open(CERT_FILE, "wt") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
    with open(KEY_FILE, "wt") as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

In [81]:
cert_gen(emailAddress="A01750443@tec.mx",
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "privateJERS.key",
    CERT_FILE="CertificadoJERS.crt")

In [62]:
import OpenSSL.crypto 
cert = OpenSSL.crypto.load_certificate(
    OpenSSL.crypto.FILETYPE_PEM, 
    open('CertificadoJERS.crt').read()
)
cert.get_notBefore()

b'20220407134553Z'

In [77]:
print(cert.get_signature_algorithm()) #Con qué algoritmo fue firmado
print(cert.get_notAfter()) #fecha de expiración
print(cert.get_pubkey())
print(cert.get_subject()) # Datos sobre el certificado

b'sha512WithRSAEncryption'
b'20230407134553Z'
<X509Name object '/C=MX/ST=Nuevo Le\xC3\xB3n/L=Monterrey/O=Tecnol\xC3\xB3gico de Monterrey/OU=organizationUnitName/CN=tec.mx/emailAddress=A01750443@tec.mx'>
